# Winepi

In [1]:
from episode_miner import EventTagger, EventText, EventSequence

Use [EventTagger](docs/EventTagger.ipynb) and [EventText](docs/EventText.ipynb) to find the events in text.

In [2]:
event_vocabulary = [{'term': 'üks'}, 
                    {'term': 'kaks'}]    
event_tagger = EventTagger(event_vocabulary, search_method='naive', conflict_resolving_strategy='ALL')
event_text = EventText('üks kaks üks kolm üks kaks üks kaks kolm kaks kolm üks kaks', event_tagger=event_tagger)

Now extract from ``event_text`` event sequence. That is the main input for the Winepi algorithm.

In [3]:
event_sequence = EventSequence(event_text=event_text, classificator='term', determine_event_time_by='char')

print(event_sequence.start)
print(event_sequence.end)
print(event_sequence.sequence_of_events)

0
34
[(üks, 0), (kaks, 2), (üks, 4), (üks, 11), (kaks, 13), (üks, 15), (kaks, 17), (kaks, 24), (üks, 31), (kaks, 33)]


Apply Winepi algorithm to find frequent serial episodes in event sequence.

...